# 1.加载数据
    pd.read_sql 优化问题 https://yq.aliyun.com/articles/607391
    
# 2.处理数据

In [223]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 1.1 从csv读取数据

In [224]:
def load_data(filename):
    print("\n正在加载数据......")
    data = pd.read_csv(filename)
    print("数据加载完成!!!\n")
    return data

data = load_data('ratings.csv')


正在加载数据......
数据加载完成!!!



## 1.2 划分测试集和训练集 

In [72]:

def split_data(data):
    print("\n正在划分数据集......")
    train_data, test_data = train_test_split(data, test_size=0.2)
    print("测试集{}条\t 训练集{}条".format(test_data.count()[0], train_data.count()[0]))
    print("划分数据集完成!!!\n")
    return train_data, test_data

train_data, test_data = split_data(data)


正在划分数据集......
测试集20168条	 训练集80668条
划分数据集完成!!!



## 2.1 构建movie_user 表 {}

In [84]:
train_data.head()

,userId,movieId,rating,timestamp
22044,144,1682,2.5,1137323966
44963,298,7361,2.5,1447518020
28012,192,110,3.0,835128955
59499,387,1562,1.5,1095041022
56516,375,1923,4.5,1225314346


In [173]:
pivot_movie_user = pd.pivot_table(train_data, index=["movieId","userId"], values="rating")
print('ratings 的 值 {} '.format(list(pivot_movie_user.values)[0][0]))
pivot_movie_user.query('movieId == 1')

ratings 的 值 4.0 


rating
movieId userId        
1       1          4.0
        7          4.5
        17         4.5
        18         3.5
        21         3.5
...                ...
        604        3.0
        605        4.0
        607        4.0
        608        2.5
        609        3.0

[175 rows x 1 columns]

In [129]:
pd.pivot_table(train_data, index=["movieId","userId"], values="rating").iloc[0:1]

,,rating
movieId,userId,
1,1,4.0


# 查看哪些电影没有被爬取 

In [390]:


import pymysql
import time
con = pymysql.connect(host='127.0.0.1', user='root', password='wozhiai0',db='movies')
Ratings = pd.read_sql("select * from rate", con)
# pivot_movie_user = pd.pivot_table(Ratings, index=["movie_id","user_id"], values="star")
# pivot_user_movie = pd.pivot_table(Ratings, index="user_id",columns='movie_id', values="star")
ratings_movies = list(Ratings.movie_id.unique())
Movies = pd.read_sql("select * from info", con)
movies = list(Movies.id.unique())
print("ratings中的movieId 有{}条 而Info的movieId 有{}条 相差{}条".format(len(ratings_movies), len(Movies), len(ratings_movies)-len(Movies)))

r_m = list(set(ratings_movies).difference(set(movies)))
for i in r_m:
    print(i,end='')
    print()
print(len(r_m))
print('以上数据需要重新爬取')

ratings中的movieId 有50084条 而Info的movieId 有49428条 相差656条
1916936
4309001
1302541
1875985
19963931
3162140
1302557
21340191
26378296
27099199
1302604
26247281
1982581
1441915
2998425
1302698
4882608
4882609
4882610
4882611
4882613
30351548
4882625
4882629
1433797
3670218
10461388
4882640
25698513
4751571
2990294
26779885
26886387
30179572
2130165
4833542
27107596
3801357
3088667
26870044
3735839
26935584
1401124
1401129
3408170
1532205
2081071
2294068
1302836
1827136
1294666
3121493
3735925
2376071
1401223
25813403
4227486
25813409
20505003
5202353
26354098
25780664
24699321
1950139
25960901
20259277
30155222
6037975
5980630
25919982
26640888
5194245
1303053
25764369
21332521
24699451
4227664
24863312
10543699
27026013
3736158
11584101
1761920
1417868
1401504
5120686
11584182
2294458
3228352
27173571
27124420
2155210
2032341
26878686
5382879
1868513
4236014
3564292
2032389
4711220
2286418
1303378
3998551
4219760
4219762
4219763
4219764
1917852
4842410
3335115
26870741
26264547
26805230
389

2070452
3291063
3987383
25761720
1308597
2160575
1898438
20445130
30152651
25802702
1841109
5339095
30300125
26900447
2340836
5339111
1849330
26834930
5339124
34461683
25769974
4896760
26785787
2160639
26785792
27195401
10811401
3659791
1308688
26990609
6772757
10549274
5290011
26843165
27179038
1472548
10762279
10786855
6961193
26417192
1292327
1292331
4913201
26892339
2340919
2070586
2357306
5265469
25704510
1300543
7051328
26712129
1300545
1300557
26589263
5208150
34854999
5380185
3725401
26654811
34429020
2340957
26687582
2349145
1308773
2381927
2340969
26925164
2381933
34855021
26761328
1300599
2283642
25884794
26425468
10459268
25925764
4749446
26851465
11589770
1308810
26917005
26581134
1759375
3381390
26851469
26581142
3037336
5159064
3987608
3791012
25884841
4921513
26712235
30292142
1308860
2078916
26826953
1292490
1300689
27597015
1947868
4749534
3619040
1292520
2160877
3717360
26728689
1849591
26253559
24705272
20429048
33159421
1472771
26392843
11540750
26884366
26671375
2

# 数据库需要再爬取的电影信息条目 

In [248]:
import pymysql
import time
con = pymysql.connect(host='139.196.137.235', user='root', password='wozhiai0',db='movies')
Movies = pd.read_sql("select id from info", con)
ratings = pd.read_sql("select movie_id from rate", con)
movies = list(Movies.id.unique())
id1 = list(ratings.movie_id.unique())
dif = list(set(id1).difference(set(movies)))
for i in dif:
    print(i)

1916936
4309001
1302541
1875985
19963931
3162140
1302557
21340191
26378296
27099199
1302604
26247281
1982581
1441915
2998425
1302698
4882608
4882609
4882610
4882611
4882613
30351548
4882625
4882629
1433797
3670218
10461388
4882640
25698513
4751571
2990294
26779885
26886387
30179572
2130165
4833542
27107596
3801357
3088667
26870044
3735839
26935584
1401124
1401129
3408170
1532205
2081071
2294068
1302836
1827136
1294666
3121493
3735925
2376071
1401223
25813403
4227486
25813409
20505003
5202353
26354098
25780664
24699321
1950139
25960901
20259277
30155222
6037975
5980630
25919982
26640888
5194245
1303053
25764369
21332521
24699451
4227664
24863312
10543699
27026013
3736158
11584101
1761920
1417868
1401504
5120686
11584182
2294458
3228352
27198145
1401538
27173571
27124420
2155210
2032341
26878686
5382879
1868513
4236014
3564292
2032389
4711220
2286418
1303378
3998551
4219760
4219762
4219763
4219764
1917852
4842410
3335115
26870741
26264547
26805230
3892207
1401849
1885180
2245630
6038543


4896760
26785787
2160639
26785792
27195401
10811401
3659791
1308688
26990609
6772757
10549274
5290011
26843165
27179038
1472548
10762279
10786855
6961193
26417192
1292327
1292331
4913201
26892339
2340919
2070586
2357306
5265469
25704510
1300543
7051328
26712129
1300545
1300557
26589263
5208150
34854999
5380185
3725401
26654811
34429020
2340957
26687582
2349145
1308773
2381927
2340969
26925164
2381933
34855021
26761328
1300599
2283642
25884794
26425468
10459268
25925764
4749446
26851465
11589770
1308810
26917005
26581134
1759375
3381390
26851469
26581142
3037336
5159064
3987608
3791012
25884841
4921513
26712235
30292142
1308860
2078916
26826953
1292490
1300689
27597015
1947868
4749534
3619040
1292520
2160877
3717360
26728689
1849591
26253559
24705272
20429048
33159421
1472771
26392843
11540750
26884366
26671375
27007255
1308952
26908955
3619100
4315420
27064608
26442029
27171119
10484016
3733810
1464633
26335546
4798779
27089211
1308988
27154751
10484037
2996549
22559053
1866061
1898830

In [249]:
len(dif)

1734

In [250]:
con = pymysql.connect(host='127.0.0.1', user='root', password='wozhiai0',db='movies')
cursor = con.cursor()
cursor.execute('select * from rate')
D = cursor.fetchall()
print(type(D))

<class 'tuple'>


In [262]:
# 每一条观看纪录 rate[1]:userid rate[2]:movie_id rate[3]:star
movie_dic = {}
for rate in D:
    u_id = rate[1]
    m_id = rate[2]
    movie_dic.setdefault(m_id,[])
    movie_dic[m_id].append(u_id)

In [291]:
# 根据movie_dic 得到用户-用户观看相同电影的count
u_u_dic = {} 
for m_id,u_list in movie_dic.items():
    for u in u_list:
        u_u_dic.setdefault(u,{})
        for v in u_list:
            if (u==v):
                continue
            u_u_dic[u].setdefault(v,0)
            u_u_dic[u][v] += 1

In [308]:
[i[0] for i in sorted(u_u_dic['beijinglife'].items(), key=itemgetter(1), reverse=True)[0:10]]

['1019579',
 'YUYII',
 '56906475',
 '13129748',
 'movieview',
 'zuozuoqing',
 'joestarfang',
 'haotusiji',
 'sensitivebastar',
 'jamesynaruto']

In [309]:
u_nearest_k = {}
for user,user_dic in u_u_dic.items():
    u_nearest_k[user] = [i[0] for i in sorted(u_u_dic[user].items(), key=itemgetter(1), reverse=True)[0:10]]

In [310]:
u_nearest_k['beijinglife']

['1019579',
 'YUYII',
 '56906475',
 '13129748',
 'movieview',
 'zuozuoqing',
 'joestarfang',
 'haotusiji',
 'sensitivebastar',
 'jamesynaruto']

In [373]:

cosine_similarity(np.array([[1, 2]]), np.array([[2, 1]]))

array([[0.8]])

In [385]:
mean = Ratings.groupby(by='user_id',as_index=False)['star'].mean()
Ratings_mean = pd.merge(Ratings,mean,on='user_id')
Ratings_mean['adg'] = Ratings_mean['star_x']-Ratings_mean['star_y']
user_item_matrix = pd.pivot_table(Ratings_mean,index='user_id',columns='movie_id',values='adg')
user_item_matrix = user_item_matrix.fillna(user_item_matrix.mean(axis=0))

In [379]:
from sklearn.metrics.pairwise import cosine_similarity
u1 = list(test.loc['beijinglife'])
u2 = list(test.loc['1019579'])
cosine_similarity([u1],[u2])

In [394]:
### 计算用户相似度 (缺陷：可能两个用户的其他电影数据 比如没有看过的越多，他们维度一样。改进：把没看过的不比较)
from sklearn.metrics.pairwise import cosine_similarity
def cal_user_similarity(user, user_item_matrix, u_nearest_k):
    user_similaruser_value = {}
    u1_data = list(user_item_matrix.loc[user])
    for u2 in u_nearest_k[user]:
        u2_data = list(user_item_matrix.loc[u2])
        similarity = cosine_similarity([u1_data],[u2_data])
        user_similaruser_value[u2] = list(similarity)[0][0]
        #print("user_id:{} 的相似度:{}".format(u2,similarity))
    return user_similaruser_value

cal_user_similarity('beijinglife', user_item_matrix, u_nearest_k)

{'1019579': 0.9533015430144127,
 '13129748': 0.9506799572204316,
 '56906475': 0.9152035756030206,
 'YUYII': 0.9748429484112268,
 'haotusiji': 0.975261096762053,
 'jamesynaruto': 0.9468857076496549,
 'joestarfang': 0.9700840989522374,
 'movieview': 0.9530068339757143,
 'sensitivebastar': 0.9727902137439852,
 'zuozuoqing': 0.9657537371402924}

In [430]:
import time
from operator import itemgetter
from sklearn.metrics.pairwise import cosine_similarity

## 程序运行时间
def calculate_function_run_time(func):
    def call_fun(*args, **kwargs):
        start_time = time.time()
        f = func(*args, **kwargs)
        end_time = time.time()
        print('%s() 运行时间为：%s s' % (func.__name__, int(end_time - start_time)))
        return f
    return call_fun


import pymysql
import pandas as pd
@calculate_function_run_time
def load_data(filename=None):
    # read_sql 方式   15s
    print('开始加载用户行为数据,生成了行为dataframe')
    con = pymysql.connect(host='127.0.0.1', user='root', password='wozhiai0',db='movies')
    Ratings = pd.read_sql("select * from rate", con)
    return Ratings

    # 查询数据库方式
    # con = pymysql.connect(host='127.0.0.1', user='root', password='wozhiai0',db='movies')
    # cursor = con.cursor()
    # cursor.execute('select * from rate')
    # D = cursor.fetchall()
    # print(type(D))
    # return D

## 建立倒查表
@calculate_function_run_time
def create_movie_dic(data):
    # 每一条评价纪录 生成movie_dic = {'12':['beijinglife','yxf']...}
    print('开始生成{电影:所观看的用户列表,...}movie_dic')
    movie_dic = {}
    for index,rate in data.iterrows():
        u_id = rate[1]
        m_id = rate[2]
        movie_dic.setdefault(m_id,[])
        movie_dic[m_id].append(u_id)
    return movie_dic

@calculate_function_run_time
def create_user_user_dic(movie_dic):
    # 根据movie_dic 得到用户-用户观看相同电影的count
    print('开始生成{用户1:{用户2:10}, 用户2:{用户1:10}} 倒查表')
    u_u_dic = {} 
    for m_id,u_list in movie_dic.items():
        for u in u_list:
            u_u_dic.setdefault(u,{})
            for v in u_list:
                if (u==v):
                    continue
                u_u_dic[u].setdefault(v,0)
                u_u_dic[u][v] += 1
    return u_u_dic

## 寻找最近的k个用户
@calculate_function_run_time
def get_n_nearest_nei(u_u_dic, k=5):
    print('开始生成{用户1:[最相似k个用户列表]} 最相似用户表')
    u_nearest_k = {}
    for user,user_dic in u_u_dic.items():
        u_nearest_k[user] = [i[0] for i in sorted(u_u_dic[user].items(), key=itemgetter(1), reverse=True)[0:k]]
    return u_nearest_k

## 生成user-item矩阵
@calculate_function_run_time
def create_user_item_matrix(Ratings):
    print('开始生成用户-电影矩阵')
    mean = Ratings.groupby(by='user_id',as_index=False)['star'].mean()
    Ratings_mean = pd.merge(Ratings,mean,on='user_id')
    # 添加一列 偏好程度
    Ratings_mean['adg'] = Ratings_mean['star_x']-Ratings_mean['star_y']
    user_item_matrix = pd.pivot_table(Ratings_mean,index='user_id',columns='movie_id',values='adg')
    # Nan补充为电影平均分
    user_item_matrix = user_item_matrix.fillna(user_item_matrix.mean(axis=0))
    return user_item_matrix

# 计算某个user的相似用户相似度
def cal_user_similarity(user, user_item_matrix, u_nearest_k):
    print('开始计算某用户最相似的k个用户的相似度')
    u1_data = list(user_item_matrix.loc[user])
    user_similaruser_value = {}
    for u2 in u_nearest_k[user]:
        u2_data = list(user_item_matrix.loc[u2])
        similarity = cosine_similarity([u1_data],[u2_data])
        user_similaruser_value[u2] = list(similarity)[0][0]
        #print("user_id:{} 的相似度:{}".format(u2,similarity))
    return user_similaruser_value



In [407]:
data = load_data()

load_data() 运行时间为：14 s


In [425]:
movie_dic = create_movie_dic(data)

create_movie_dic() 运行时间为：124 s


In [426]:
u_u_dic = create_user_user_dic(movie_dic)


create_user_user_dic() 运行时间为：52 s


In [427]:
u_nerest_k = get_n_nearest_nei(u_u_dic, k=5)

get_n_nearest_nei() 运行时间为：8 s


In [428]:
user_item_matrix = create_user_item_matrix(data)

create_user_item_matrix() 运行时间为：86 s


In [463]:
user_similaruser_value = cal_user_similarity('123456', user_item_matrix, u_nearest_k)

开始计算某用户最相似的k个用户的相似度


In [464]:
user_similaruser_value

{'1991817': 0.9895372741648217,
 '43645225': 0.9947624679790645,
 'AdaDream': 0.9856941097018584,
 'YUYII': 0.9902031705302823,
 'bangle': 0.9913327106196649,
 'beijinglife': 0.9847582262202648,
 'haotusiji': 0.9899354778219368,
 'joestarfang': 0.9862788030950504,
 'vanille': 0.9929049879945951,
 'zuozuoqing': 0.979826482532}

In [470]:
def recommend(data, user, user_similaruser_value):
    print('获得电影预测评分')
    movie_seen_by_user = data['movie_id'][data['user_id']==user].tolist()
    movie_seen_by_similar = set()
    for u,v in user_similaruser_value.items():
        print(data['movie_id'][data['user_id']==u].tolist())
        #movie_seen_by_similar_str += ','.join()
    #print(movie_seen_by_similar_str)

In [473]:
data['movie_id'][data['user_id']=='1991817'].tolist()

[1938312,
 1292219,
 1962319,
 1473297,
 1307793,
 1459323,
 2043497,
 1292331,
 1308480,
 1302391,
 1421278,
 1806672,
 1867822,
 1308575,
 1291999,
 1759371,
 1292000,
 1306792,
 1304648,
 1788392,
 1307649,
 1291546,
 1308076,
 1316559,
 2300091,
 1464548,
 1479313,
 1765127,
 1292289,
 1292449,
 1418834,
 1300528,
 1291828,
 1292220,
 1401477,
 1299661,
 1437684,
 1292339,
 1292792,
 1418070,
 1291545,
 1824951,
 1914831,
 1309052,
 1875313,
 1308748,
 1422081,
 1307766,
 1305182,
 2157130,
 2381939,
 2155187,
 1931974,
 1297102,
 1292720,
 1419969,
 1301853,
 2136071,
 1298974,
 1293596,
 1309015,
 1295644,
 1949811,
 1820145,
 1292274,
 1308749,
 1303021,
 1298624,
 1291870,
 1482394,
 1727557,
 1306123,
 1308747,
 1291550,
 1766086,
 2215897,
 2262116,
 2282942,
 2210031,
 1907464,
 1291555,
 1304708,
 1759912,
 1418019,
 1961963,
 1306795,
 1937502,
 2084227,
 1305279,
 1369747,
 2326215,
 1294408,
 1292224,
 1292212,
 1300353,
 1899767,
 1292052,
 1826652,
 1478926,
 1304613,


In [482]:
a

{1}

In [484]:
a = [1,2,3]
a

[1, 2, 3]